In [1]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [13]:
## CREATE HOLE DATA 
## DON'T NEED TO RUN THIS EVERY TIME

sg_data = pd.read_csv('C:/Users/Luke/OneDrive/Research/sg_data/hole_sg_by_shot_category_2018.csv', sep = ';', encoding = 'ISO-8859-1')

sg_data[['approach','close','tee','putt']]= pd.get_dummies(sg_data['sg_cat_name'])
sg_data['course_hole'] = sg_data.apply(lambda row: (row['course_name'],row['hole']), axis = 1)
sg_data['h_idx'] = sg_data.course_hole.astype("category").cat.codes
sg_data['p_idx'] = sg_data.player.astype("category").cat.codes
sg_data['plyr_hole'] = sg_data.apply(lambda row: (row['p_idx'],row['h_idx']), axis = 1)

sg_data['SG_appr']= sg_data['approach']* sg_data['sg']
sg_data['SG_short']= sg_data['close']* sg_data['sg']
sg_data['SG_drive']= sg_data['tee']* sg_data['sg']
sg_data['SG_putt']= sg_data['putt']* sg_data['sg']

pp_hole = pd.DataFrame(sg_data.groupby(['player','course_hole','h_idx','p_idx'])['SG_appr'].sum()).reset_index()
pp_hole['SG_short'] = pd.DataFrame(sg_data.groupby(['player','course_hole'])['SG_short'].sum()).reset_index()['SG_short']
pp_hole['SG_drive'] = pd.DataFrame(sg_data.groupby(['player','course_hole'])['SG_drive'].sum()).reset_index()['SG_drive']
pp_hole['SG_putt'] = pd.DataFrame(sg_data.groupby(['player','course_hole'])['SG_putt'].sum()).reset_index()['SG_putt']
pp_hole.to_csv('C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_2018_data.csv')

In [14]:
real_df = pd.read_csv('C:/Users/Luke/OneDrive/Research/sg_data/sg_hole_2018_data.csv')

def create_weights(df):
    weights = []
    n = len(df['p_idx'])
    for player in pd.unique(df['p_idx']):
        weights.append(len(df[df['p_idx'] == player])/n)
    return weights

scaled_weights = create_weights(real_df)
scaled_weights_df = pd.DataFrame({'plyr': pd.unique(real_df['player']),'p_idx': pd.unique(real_df['p_idx']), 'weight': scaled_weights}).sort_values(by=['p_idx']).set_index(np.arange(len(scaled_weights)))


def create_hole_weights(df):
    weights = []
    n = len(df['h_idx'])
    for course in pd.unique(df['h_idx']):
        weights.append(len(df[df['h_idx'] == course])/n)
    return weights

hole_weights = create_hole_weights(real_df)
hole_weights_df = pd.DataFrame({'hole': pd.unique(real_df['course_hole']),'h_idx': pd.unique(real_df['h_idx']), 'weight': hole_weights}).sort_values(by=['h_idx']).set_index(np.arange(len(hole_weights)))


scaled_weights_df.to_csv('C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_plyr_weights.csv')
hole_weights_df.to_csv('C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_hole_weights.csv')

In [16]:
nodsr_p_sg_ = pd.read_csv('C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_plyr_nodsr.csv').sort_values(by = 'player').reset_index(drop=True)
nodsr_h_sg_ = pd.read_csv('C:/Users/Luke/OneDrive/Research/sg_data/realv2_byhole_hole_nodsr.csv').sort_values(by = 'course_hole').reset_index(drop=True)

def create_weights(df):
    weights = []
    n = len(df['p_idx'])
    for player in pd.unique(df['p_idx']):
        weights.append(len(df[df['p_idx'] == player])/n)
    return weights

scaled_weights = create_weights(real_df)
scaled_weights_df = pd.DataFrame({'plyr': pd.unique(real_df['player']),'p_idx': pd.unique(real_df['p_idx']), 'weight': scaled_weights}).sort_values(by=['p_idx']).set_index(np.arange(len(scaled_weights)))
scaled_weights_df['sg_drive_mean']= nodsr_p_sg_['sg_drive']
scaled_weights_df['sg_appr_mean']= nodsr_p_sg_['sg_appr']
scaled_weights_df['sg_short_mean']= nodsr_p_sg_['sg_short']
scaled_weights_df['sg_putt_mean']= nodsr_p_sg_['sg_putt']
scaled_weights_df['sg_total_mean']= nodsr_p_sg_['sg_drive'] + nodsr_p_sg_['sg_appr'] + nodsr_p_sg_['sg_short'] +nodsr_p_sg_['sg_putt'] 


def create_hole_weights(df):
    weights = []
    n = len(df['h_idx'])
    for course in pd.unique(df['h_idx']):
        weights.append(len(df[df['h_idx'] == course])/n)
    return weights

hole_weights = create_hole_weights(real_df)
hole_weights_df = pd.DataFrame({'hole': pd.unique(real_df['course_hole']),'h_idx': pd.unique(real_df['h_idx']), 'weight': hole_weights}).sort_values(by=['h_idx']).set_index(np.arange(len(hole_weights)))

hole_weights_df['sg_drive_mean']= nodsr_h_sg_['sg_drive']
hole_weights_df['sg_appr_mean']= nodsr_h_sg_['sg_appr']
hole_weights_df['sg_short_mean']= nodsr_h_sg_['sg_short']
hole_weights_df['sg_putt_mean']= nodsr_h_sg_['sg_putt']
hole_weights_df['sg_total_mean'] = hole_weights_df['sg_drive_mean'] + hole_weights_df['sg_appr_mean'] + hole_weights_df['sg_short_mean']+ hole_weights_df['sg_putt_mean']


scaled_weights_df.to_csv('C:/Users/Luke/OneDrive/Research/sg_data/realv2_scaled_byhole_sg_plyr_weights.csv')
hole_weights_df.to_csv('C:/Users/Luke/OneDrive/Research/sg_data/realv2_scaled_byhole_sg_hole_weights.csv')